In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd() / "notebooks"))  # so we can import _utils from notebooks/

from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from _utils import get_data_dir

DATA_DIR = get_data_dir()
ROOT = get_data_dir()

In [3]:
conds = ["100_0", "80_20", "50_50"]

bio_dfs = []

for cond in conds:
    base = ROOT / cond
    
    # Use the angle-estimation file: one row per trial → one full iteration
    g = pd.read_csv(base / "angle_estimations" / "gemini-2.5-pro.csv")
    
    # Build BIO scaffolding
    g["stimID"] = g["image_id"]
    g["condition"] = cond
    g["participantID"] = "BIO"
    
    # Placeholder response column for BIO (no decisions yet)
    g["response"] = np.nan

    # Keep columns analogous to model_master, but **without** the 'decision' column
    bio_dfs.append(
        g[[
            "stimID",
            "condition",
            "side_selected",
            "cue_points",
            "line1_angle",
            "line2_angle",
            "valid_cue",
            "TP",
            "response",
            "participantID",
        ]]
    )

BIO_master = pd.concat(bio_dfs, ignore_index=True)

display(BIO_master)

,stimID,condition,side_selected,cue_points,line1_angle,line2_angle,valid_cue,TP,response,participantID
0,100,100_0,1,1,14.314827,1.921956,True,True,NaN,BIO
1,845,100_0,1,1,15.054317,4.222230,True,True,NaN,BIO
2,245,100_0,1,1,14.314827,6.508956,True,True,NaN,BIO
3,72,100_0,2,2,8.775056,15.054317,True,True,NaN,BIO
4,469,100_0,2,2,4.222230,19.885165,True,True,NaN,BIO
...,...,...,...,...,...,...,...,...,...,...
2995,698,50_50,1,2,11.159944,6.508956,False,False,NaN,BIO
2996,910,50_50,1,1,9.651833,4.986334,True,False,NaN,BIO
2997,880,50_50,2,2,4.222230,8.775056,True,False,NaN,BIO
2998,53,50_50,2,2,4.986334,11.908300,True,False,NaN,BIO


In [4]:
class BayesianIdealObserver:
    """
    Bayesian Ideal Observer (BIO) for the 2AFC target-detection task with a spatial cue.

    Assumptions (per trial):
      - Two angles: line1_angle (left), line2_angle (right).
      - Target-present TP=1 vs target-absent TP=0.
      - TP=0: both angles ~ N(mu_noise, sigma^2), cue random (0.5 left/right).
      - TP=1: one side S∈{1,2} is target side, chosen with prob 0.5.
              x_S     ~ N(mu_target, sigma^2)
              x_other ~ N(mu_noise, sigma^2)
              cue points to S with prob v (cue validity), otherwise to other side.
      - Condition determines validity v:
            50_50 -> 0.5
            80_20 -> 0.8
            100_0 -> 1.0
      - Prior P(TP=1) = prior_tp (default 0.5).

    This class:
      * Adds BIO columns (likelihood ratio, log LR, posterior, decisions) to a DataFrame.
      * Computes SDT metrics (accuracy, hit rate, fa rate, d', criterion) by condition.
    """

    def __init__(
        self,
        mu_target: float = 15.0,
        mu_noise: float = 5.0,
        sigma: float | None = 3.5,
        prior_tp: float = 0.5,
        cond_validity: dict | None = None,
    ):
        """
        Parameters
        ----------
        mu_target : float
            Mean angle for the target distribution.
        mu_noise : float
            Mean angle for the noise distribution.
        sigma : float or None
            Sensory noise standard deviation. If None, it will be estimated
            from TP=0 trials of whatever DataFrame you pass to add_bio_columns.
        prior_tp : float
            Prior probability P(TP=1).
        cond_validity : dict or None
            Mapping from condition label -> cue validity v.
            If None, defaults to: {"50_50": 0.5, "80_20": 0.8, "100_0": 1.0}.
        """
        self.mu_target = mu_target
        self.mu_noise = mu_noise
        self.sigma = sigma  # if None, will be fit from TP=0 trials
        self.prior_tp = prior_tp

        if cond_validity is None:
            cond_validity = {
                "50_50": 0.5,
                "80_20": 0.8,
                "100_0": 1.0,
            }
        self.cond_validity = cond_validity

    # ---------------------------------------------------------
    # Internal helper: estimate sigma from TP=0 trials
    # ---------------------------------------------------------
    def _estimate_sigma_from_tp0(
        self,
        df: pd.DataFrame,
        line1_col: str,
        line2_col: str,
        tp_col: str,
    ) -> float:
        tp0 = df[df[tp_col] == 0]
        all_angles = np.concatenate(
            [tp0[line1_col].to_numpy(), tp0[line2_col].to_numpy()]
        )
        sigma_hat = all_angles.std(ddof=1)
        if not np.isfinite(sigma_hat) or sigma_hat <= 0:
            sigma_hat = 1.0  # very conservative fallback
        return sigma_hat

    # ---------------------------------------------------------
    # Main method: add BIO columns to a DataFrame
    # ---------------------------------------------------------
    def add_bio_columns(
        self,
        df: pd.DataFrame,
        line1_col: str = "line1_angle",
        line2_col: str = "line2_angle",
        cue_col: str = "cue_points",
        cond_col: str = "condition",
        tp_col: str = "TP",
    ) -> pd.DataFrame:
        """
        Return a copy of df with the following added columns:

          - bio_sigma      : sigma used by the BIO
          - bio_lambda     : likelihood ratio Λ = p(x,c|TP=1)/p(x,c|TP=0)
          - bio_llr        : log(Λ)
          - bio_p_present  : posterior P(TP=1 | x1,x2,c,cond)
          - bio_decision   : 1 if BIO says "target present", else 0
        """

        out = df.copy()

        # 1. Decide sigma
        if self.sigma is None:
            sigma = self._estimate_sigma_from_tp0(out, line1_col, line2_col, tp_col)
        else:
            sigma = float(self.sigma)
        out["bio_sigma"] = sigma

        # 2. Extract angles, cue, condition
        x1 = out[line1_col].to_numpy()
        x2 = out[line2_col].to_numpy()
        cue = out[cue_col].to_numpy()  # 1 = left, 2 = right
        # map each condition to its validity v
        v = out[cond_col].map(self.cond_validity).to_numpy()

        # 3. Angle evidence: R(x) = N(x; mu_target,σ²) / N(x; mu_noise,σ²)
        #    Using the closed-form ratio for equal-variance Gaussians.
        delta_mu = self.mu_target - self.mu_noise
        offset = (self.mu_target**2 - self.mu_noise**2) / 2.0
        denom = sigma**2

        R1 = np.exp((delta_mu * x1 - offset) / denom)
        R2 = np.exp((delta_mu * x2 - offset) / denom)

        # 4. Cue likelihoods given target side
        #    For each trial:
        #      p(cue | S=1) = v if cue==1 else 1-v
        #      p(cue | S=2) = v if cue==2 else 1-v
        p_c_given_S1 = np.where(cue == 1, v, 1.0 - v)
        p_c_given_S2 = np.where(cue == 2, v, 1.0 - v)

        # 5. Likelihood ratio Λ(x1,x2,c,cond)
        #    Λ = R1 * p(c|S=1) + R2 * p(c|S=2)
        Lambda = R1 * p_c_given_S1 + R2 * p_c_given_S2

        # Numerical safety
        eps = 1e-12
        Lambda_safe = np.maximum(Lambda, eps)
        log_Lambda = np.log(Lambda_safe)

        out["bio_lambda"] = Lambda_safe
        out["bio_llr"] = log_Lambda

        # 6. Posterior with prior P(TP=1) = prior_tp
        # Posterior odds = Λ * prior_odds
        # P(TP=1 | data) = odds / (1 + odds)
        prior_tp = np.clip(self.prior_tp, 1e-6, 1 - 1e-6)
        prior_odds = prior_tp / (1.0 - prior_tp)

        log_posterior_odds = log_Lambda + np.log(prior_odds)
        posterior_odds = np.exp(log_posterior_odds)
        p_present = posterior_odds / (1.0 + posterior_odds)

        out["bio_p_present"] = p_present

        # 7. Bayes-optimal decision: TP=1 iff posterior odds > 1
        out["bio_decision"] = (log_posterior_odds > 0).astype(int)

        return out

    # ---------------------------------------------------------
    # SDT metrics
    # ---------------------------------------------------------
    @staticmethod
    def _compute_sdt_metrics(
        df: pd.DataFrame,
        decision_col: str = "bio_decision",
        tp_col: str = "TP",
    ) -> pd.Series:
        n_trials = len(df)
        n_signal = (df[tp_col] == 1).sum()
        n_noise = (df[tp_col] == 0).sum()

        hits = ((df[tp_col] == 1) & (df[decision_col] == 1)).sum()
        fas = ((df[tp_col] == 0) & (df[decision_col] == 1)).sum()
        crs = ((df[tp_col] == 0) & (df[decision_col] == 0)).sum()

        accuracy = (hits + crs) / n_trials if n_trials > 0 else np.nan

        # log-linear correction for extreme rates
        H = (hits + 0.5) / (n_signal + 1) if n_signal > 0 else np.nan
        F = (fas + 0.5) / (n_noise + 1) if n_noise > 0 else np.nan

        if np.isfinite(H) and np.isfinite(F):
            zH = norm.ppf(H)
            zF = norm.ppf(F)
            dprime = zH - zF
            criterion = -0.5 * (zH + zF)
        else:
            dprime = np.nan
            criterion = np.nan

        return pd.Series(
            {
                "n_trials": n_trials,
                "accuracy": accuracy,
                "hit_rate": H,
                "fa_rate": F,
                "dprime": dprime,
                "criterion": criterion,
            }
        )

    def compute_sdt_by_condition(
        self,
        df: pd.DataFrame,
        condition_col: str = "condition",
        decision_col: str = "bio_decision",
        tp_col: str = "TP",
    ) -> pd.DataFrame:
        """
        Group df by condition and compute SDT metrics for the BIO decisions.
        Returns a DataFrame with one row per condition.
        """
        grouped = (
            df.groupby(condition_col, as_index=False)
            .apply(lambda sub: self._compute_sdt_metrics(sub, decision_col, tp_col))
        )
        # groupby-apply with as_index=False returns condition_col as index; reset nicely:
        metrics = grouped.reset_index(drop=True)
        metrics.rename(columns={condition_col: "condition"}, inplace=True)
        metrics["who"] = "BIO"
        return metrics


In [7]:
bio = BayesianIdealObserver(
    mu_target=15.0,
    mu_noise=5.0,
    sigma=3.5,   # or None to estimate from TP=0
    prior_tp=0.5
)

bio_with_decisions = bio.add_bio_columns(BIO_master)

BIO_metrics = bio.compute_sdt_by_condition(bio_with_decisions)
display(BIO_metrics)


C:\Users\AdamR\AppData\Local\Temp\ipykernel_10480\1579459639.py:219: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda sub: self._compute_sdt_metrics(sub, decision_col, tp_col))


,condition,n_trials,accuracy,hit_rate,fa_rate,dprime,criterion,who
0,100_0,1000.0,0.920,0.931138,0.092814,2.807940,-0.080348,BIO
1,50_50,1000.0,0.886,0.895210,0.124750,2.406281,-0.051578,BIO
2,80_20,1000.0,0.886,0.897206,0.126747,2.407695,-0.061941,BIO


In [10]:
display(bio_with_decisions[250:270])

,stimID,condition,side_selected,cue_points,line1_angle,line2_angle,valid_cue,TP,response,participantID,bio_sigma,bio_lambda,bio_llr,bio_p_present,bio_decision
250,174,100_0,1,1,13.392498,6.508956,True,True,NaN,BIO,3.5,15.948837,2.769386,0.940999,1
251,548,100_0,1,1,16.735996,5.748663,True,True,NaN,BIO,3.5,244.391665,5.498772,0.995925,1
252,787,100_0,2,2,4.222230,12.652557,True,True,NaN,BIO,3.5,8.717672,2.165352,0.897095,1
253,320,100_0,1,1,13.570434,4.986334,True,True,NaN,BIO,3.5,18.442178,2.914640,0.948565,1
254,462,100_0,2,2,4.986334,11.908300,True,True,NaN,BIO,3.5,4.748344,1.557796,0.826037,1
255,657,100_0,2,2,1.921956,15.788708,True,True,NaN,BIO,3.5,112.784138,4.725476,0.991211,1
256,52,100_0,1,1,10.407711,3.456619,True,True,NaN,BIO,3.5,1.394904,0.332826,0.582447,1
257,706,100_0,2,2,5.748663,21.587886,True,True,NaN,BIO,3.5,12829.450627,9.459499,0.999922,1
258,197,100_0,1,1,10.407711,4.986334,True,True,NaN,BIO,3.5,1.394904,0.332826,0.582447,1
259,983,100_0,1,1,11.908300,2.689770,True,True,NaN,BIO,3.5,4.748344,1.557796,0.826037,1
